# Exercise 3

This notebook illustrates how we can use embeddings in Machine Learning tasks.

As always, we first import neccesary modules. We also get our data.

In [1]:
#!pip install embeddingvectorizer    # you need to install this module

In [2]:
# Supervised text classification

from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Embedding, LSTM, GlobalMaxPooling1D
from keras.metrics import Precision, Recall
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from sklearn.utils import shuffle

# general
import numpy as np
import re
# word embedding stuff
import gensim
import gensim.downloader as api
from gensim.similarities import SoftCosineSimilarity, SparseTermSimilarityMatrix, WordEmbeddingSimilarityIndex
from gensim.corpora import Dictionary

# data
from courseutils import get_review_data

# lets get more output
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

/usr/local/lib/python3.8/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [ ]:
# we need to have gensim 4.0 or higher - there are quite some extensive syntax changes, see links below
# (older version of this notebook was for gensim 3 -- hope I migrated everything correctly...)
# https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4
# https://github.com/RaRe-Technologies/gensim/wiki/Using-Gensim-Embeddings-with-Keras-and-Tensorflow
print(gensim.__version__)
assert gensim.__version__[0] =='4'

In [3]:
# get data
reviews_train, reviews_test, y_train, y_test = get_review_data()

reviews_train, y_train = shuffle(reviews_train, y_train, random_state=42)
reviews_test, y_test = shuffle(reviews_test, y_test, random_state=42)

# get word embedding model

# pretrained:
# wv = api.load('word2vec-google-news-300')
wv = api.load("glove-wiki-gigaword-300")

# or our own:
#wv = gensim.models.Word2Vec.load("mymodel").wv

Using cached file reviewdata.pickle.bz2


2021-04-29 19:43:53,090 : INFO : loading projection weights from /home/damian/gensim-data/glove-wiki-gigaword-300/glove-wiki-gigaword-300.gz
2021-04-29 19:44:56,674 : INFO : KeyedVectors lifecycle event {'msg': 'loaded (400000, 300) matrix of type float32 from /home/damian/gensim-data/glove-wiki-gigaword-300/glove-wiki-gigaword-300.gz', 'binary': False, 'encoding': 'utf8', 'datetime': '2021-04-29T19:44:56.666398', 'gensim': '4.0.1', 'python': '3.8.5 (default, Jan 27 2021, 15:41:15) \n[GCC 9.3.0]', 'platform': 'Linux-5.4.0-72-generic-x86_64-with-glibc2.29', 'event': 'load_word2vec_format'}


In [4]:
api.info(name_only=True)['models']

['fasttext-wiki-news-subwords-300',
 'conceptnet-numberbatch-17-06-300',
 'word2vec-ruscorpora-300',
 'word2vec-google-news-300',
 'glove-wiki-gigaword-50',
 'glove-wiki-gigaword-100',
 'glove-wiki-gigaword-200',
 'glove-wiki-gigaword-300',
 'glove-twitter-25',
 'glove-twitter-50',
 'glove-twitter-100',
 'glove-twitter-200',
 '__testing_word2vec-matrix-synopsis']

In [5]:
len(reviews_train) == len(y_train)

True

In [6]:
type(reviews_train)

list

In [7]:
reviews_train[:3]

['Dumb is as dumb does, in this thoroughly uninteresting, supposed black comedy. Essentially what starts out as Chris Klein trying to maintain a low profile, eventually morphs into an uninspired version of "The Three Amigos", only without any laughs. In order for black comedy to work, it must be outrageous, which "Play Dead" is not. In order for black comedy to work, it cannot be mean spirited, which "Play Dead" is. What "Play Dead" really is, is a town full of nut jobs. Fred Dunst does however do a pretty fair imitation of Billy Bob Thornton\'s character from "A Simple Plan", while Jake Busey does a pretty fair imitation of, well, Jake Busey. - MERK',
 "I dug out from my garage some old musicals and this is another one of my favorites. It was written by Jay Alan Lerner and directed by Vincent Minelli. It won two Academy Awards for Best Picture of 1951 and Best Screenplay. The story of an American painter in Paris who tries to make it big. Nina Foch is a sophisticated lady of means and

In [8]:
y_train[:20]

['neg',
 'pos',
 'neg',
 'pos',
 'pos',
 'neg',
 'neg',
 'pos',
 'neg',
 'neg',
 'pos',
 'neg',
 'neg',
 'pos',
 'neg',
 'pos',
 'pos',
 'pos',
 'neg',
 'neg']

# Keras

## A simple neural network

In [9]:
VALIDATION_SIZE = 2500
np.random.seed(666)



In [10]:
def encodeY(Y):
    '''create one-hot (dummies) for output, see also https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/
    encode class values as integers
    '''
    encoder = LabelEncoder()
    encoder.fit(Y)
    encoded_Y = encoder.transform(Y)
    dummy_y = np_utils.to_categorical(encoded_Y)
    return dummy_y

In [11]:
encodeY(['aa','bb','aa','cc','aa','cc'])

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.]], dtype=float32)

In [12]:
vectorizer = CountVectorizer(stop_words='english')
X_train = vectorizer.fit_transform(reviews_train)
X_test = vectorizer.transform(reviews_test)
X_test.sort_indices()
X_train.sort_indices()

input_dim = X_train.shape[1]  # Number of features

y_train_int = encodeY(y_train)[:,0]
y_test_int = encodeY(y_test)[:,0]

numberoflabels = 1

In [13]:
input_dim

74538

In [14]:
model = Sequential()
model.add(Dense(300, input_dim=input_dim, activation='relu'))
#model.add(layers.Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
           optimizer='adam', 
            metrics=['accuracy', Precision(), Recall()])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 300)               22361700  
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 301       
Total params: 22,362,001
Trainable params: 22,362,001
Non-trainable params: 0
_________________________________________________________________


In [15]:
history = model.fit(X_train[:-VALIDATION_SIZE], y_train_int[:-VALIDATION_SIZE],
                     epochs=5,
                     verbose=True,
                     validation_data=(X_train[-VALIDATION_SIZE:], y_train_int[-VALIDATION_SIZE:]))

_, acc, precision, recall = model.evaluate(X_test, y_test_int)
print(f"Accuracy: {acc:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}")

Epoch 1/5
704/704 [==============================] - 63s 89ms/step - loss: 0.3880 - accuracy: 0.8328 - precision: 0.8317 - recall: 0.8426 - val_loss: 0.2673 - val_accuracy: 0.8904 - val_precision: 0.8946 - val_recall: 0.8845
Epoch 2/5
704/704 [==============================] - 63s 90ms/step - loss: 0.0855 - accuracy: 0.9706 - precision: 0.9705 - recall: 0.9708 - val_loss: 0.3811 - val_accuracy: 0.8788 - val_precision: 0.8776 - val_recall: 0.8797
Epoch 3/5
704/704 [==============================] - 66s 94ms/step - loss: 0.0200 - accuracy: 0.9939 - precision: 0.9938 - recall: 0.9941 - val_loss: 0.5015 - val_accuracy: 0.8800 - val_precision: 0.8828 - val_recall: 0.8757
Epoch 4/5
704/704 [==============================] - 60s 86ms/step - loss: 0.0043 - accuracy: 0.9995 - precision: 0.9993 - recall: 0.9998 - val_loss: 0.6406 - val_accuracy: 0.8776 - val_precision: 0.8719 - val_recall: 0.8845
Epoch 5/5
782/782 [==============================] - 2s 3ms/step - loss: 0.8810 - accuracy: 0.8515 -

## A model with a second layer

In [16]:
model = Sequential()
model.add(Dense(300, input_dim=input_dim, activation='relu'))
model.add(Dense(300, activation='relu'))
model.add(Dense(300, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
           optimizer='adam', 
            metrics=['accuracy', Precision(), Recall()])
model.summary()

history = model.fit(X_train[:-VALIDATION_SIZE], y_train_int[:-VALIDATION_SIZE],
                     epochs=5,
                     verbose=True,
                     validation_data=(X_train[-VALIDATION_SIZE:], y_train_int[-VALIDATION_SIZE:]))

_, acc, precision, recall = model.evaluate(X_test, y_test_int)
print(f"Accuracy: {acc:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}")

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 300)               22361700  
_________________________________________________________________
dense_3 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_4 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 301       
Total params: 22,542,601
Trainable params: 22,542,601
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
704/704 [==============================] - 60s 83ms/step - loss: 0.3887 - accuracy: 0.8294 - precision_1: 0.8277 - recall_1: 0.8429 - val_loss: 0.2678 - val_accuracy: 0.8856 - val_precision_1: 0.8810 - val_recall_1: 0.8909
Epoch 2

In [17]:
model = Sequential()
model.add(Dense(300, input_dim=input_dim, activation='relu'))
model.add(Dense(300, activation='relu'))
model.add(Dense(300, activation='relu'))
model.add(Dense(300, activation='relu'))

model.add(Dense(300, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
           optimizer='adam', 
            metrics=['accuracy', Precision(), Recall()])
model.summary()

history = model.fit(X_train[:-VALIDATION_SIZE], y_train_int[:-VALIDATION_SIZE],
                     epochs=5,
                     verbose=True,
                     validation_data=(X_train[-VALIDATION_SIZE:], y_train_int[-VALIDATION_SIZE:]))

_, acc, precision, recall = model.evaluate(X_test, y_test_int)
print(f"Accuracy: {acc:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}")

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 300)               22361700  
_________________________________________________________________
dense_7 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_8 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_9 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_10 (Dense)             (None, 300)               90300     
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 301       
Total params: 22,723,201
Trainable params: 22,723,201
Non-trainable params: 0
__________________________________________

## Convolutional Network

In [18]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

embedding_dim = 300

# Tokenize words
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(reviews_train)
X_train = tokenizer.texts_to_sequences(reviews_train)
X_test = tokenizer.texts_to_sequences(reviews_test)

# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

# Pad sequences with zeros
maxlen = len(max(X_train, key=len)) # never truncate -- alternatively, set max length to lower value 
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [19]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(Conv1D(embedding_dim, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(300, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',  Precision(), Recall()])
print(model.summary())


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1973, 300)         26574900  
_________________________________________________________________
conv1d (Conv1D)              (None, 1969, 300)         450300    
_________________________________________________________________
global_max_pooling1d (Global (None, 300)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 300)               90300     
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 301       
Total params: 27,115,801
Trainable params: 27,115,801
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
history = model.fit(X_train[:-VALIDATION_SIZE], y_train_int[:-VALIDATION_SIZE], 
          epochs=3, verbose=True,
          validation_data=(X_train[-VALIDATION_SIZE:], y_train_int[-VALIDATION_SIZE:]))

Epoch 1/3
704/704 [==============================] - 1255s 2s/step - loss: 0.4743 - accuracy: 0.7399 - precision_3: 0.7491 - recall_3: 0.7214 - val_loss: 0.2497 - val_accuracy: 0.8992 - val_precision_3: 0.8806 - val_recall_3: 0.9230
Epoch 2/3
704/704 [==============================] - 1254s 2s/step - loss: 0.1337 - accuracy: 0.9519 - precision_3: 0.9513 - recall_3: 0.9508 - val_loss: 0.2723 - val_accuracy: 0.8896 - val_precision_3: 0.8572 - val_recall_3: 0.9342
Epoch 3/3
704/704 [==============================] - 1327s 2s/step - loss: 0.0364 - accuracy: 0.9894 - precision_3: 0.9879 - recall_3: 0.9910 - val_loss: 0.2806 - val_accuracy: 0.9092 - val_precision_3: 0.9208 - val_recall_3: 0.8949


In [21]:
_, acc, precision, recall = model.evaluate(X_test, y_test_int)
print(f"Accuracy: {acc:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}")

782/782 [==============================] - 373s 477ms/step - loss: 0.3209 - accuracy: 0.8963 - precision_3: 0.8981 - recall_3: 0.8940
Accuracy: 0.90, Precision: 0.90, Recall: 0.89


## Pretrained embeddings

In [41]:
# from tensorflow.keras.layers import Embedding

def gensim_to_keras_embedding(wv, train_embeddings=False):
    """Get a Keras 'Embedding' layer with weights set from Word2Vec model's learned word embeddings.

    Parameters
    ----------
    train_embeddings : bool
        If False, the returned weights are frozen and stopped from being updated.
        If True, the weights can / will be further updated in Keras.

    Returns
    -------
    `keras.layers.Embedding`
        Embedding layer, to be used as input to deeper network layers.

    """
    keyed_vectors = wv  # structure holding the result of training
    weights = wv.vectors  # vectors themselves, a 2D numpy array    
    index_to_key = wv.index_to_key  # which row in `weights` corresponds to which word?

    layer = Embedding(
        input_dim=weights.shape[0],
        output_dim=weights.shape[1],
        weights=[weights],
        trainable=train_embeddings,
    )
    return layer

In [42]:
embedding_layer = gensim_to_keras_embedding(wv, train_embeddings=True)
input_dim = (len(X_train[:-VALIDATION_SIZE]), 300)

In [43]:
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(embedding_dim, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(300, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',  Precision(), Recall()])
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 300)         120000000 
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 300)         450300    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 300)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 300)               90300     
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 301       
Total params: 120,540,901
Trainable params: 120,540,901
Non-trainable params: 0
_________________________________________________________________
None


In [45]:
history = model.fit(X_train[:-VALIDATION_SIZE], y_train_int[:-VALIDATION_SIZE], 
          epochs=3, verbose=True,
          validation_data=(X_train[-VALIDATION_SIZE:], y_train_int[-VALIDATION_SIZE:]))

Epoch 1/3
704/704 [==============================] - 2256s 3s/step - loss: 0.3684 - accuracy: 0.8322 - precision_5: 0.8390 - recall_5: 0.8222 - val_loss: 0.2792 - val_accuracy: 0.8824 - val_precision_5: 0.8846 - val_recall_5: 0.8789
Epoch 2/3
704/704 [==============================] - 2211s 3s/step - loss: 0.1562 - accuracy: 0.9430 - precision_5: 0.9431 - recall_5: 0.9429 - val_loss: 0.2957 - val_accuracy: 0.8836 - val_precision_5: 0.8551 - val_recall_5: 0.9230
Epoch 3/3
704/704 [==============================] - 2109s 3s/step - loss: 0.0519 - accuracy: 0.9828 - precision_5: 0.9833 - recall_5: 0.9824 - val_loss: 0.3735 - val_accuracy: 0.8900 - val_precision_5: 0.8977 - val_recall_5: 0.8797


In [46]:
_, acc, precision, recall = model.evaluate(X_test, y_test_int)
print(f"Accuracy: {acc:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}")

782/782 [==============================] - 448s 573ms/step - loss: 0.3731 - accuracy: 0.8848 - precision_5: 0.8941 - recall_5: 0.8730
Accuracy: 0.88, Precision: 0.89, Recall: 0.87


In [ ]:
# too much memory requirements below

In [ ]:
'''
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(35))
#model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',  Precision(), Recall()])
print(model.summary())
'''

## LSTM

More examples: https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(LSTM(100))
model.add(Dense(units=1, activation='sigmoid'))
 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', Recall(), Precision()])
print(model.summary())



In [ ]:
model.fit(X_train[:-VALIDATION_SIZE], y_train_int[:-VALIDATION_SIZE], 
          epochs=3, batch_size=128, verbose=1, 
          validation_data=(X_train[-VALIDATION_SIZE:], y_train_int[-VALIDATION_SIZE:]))

In [ ]:
_, acc, precision, recall = model.evaluate(X_test, y_test_int)
print(f"Accuracy: {acc:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}")

In [ ]:
'''
history = model.fit(X_train[:-VALIDATION_SIZE], y_train_int[:-VALIDATION_SIZE], 
          epochs=5, verbose=True,
          validation_data=(X_train[-VALIDATION_SIZE:], y_train_int[-VALIDATION_SIZE:]))
'''